In [ ]:
import duckdb
import xml.etree.ElementTree as ET

In [ ]:
# Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [19]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = []
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            paragraphs_text.append(p.text.strip()) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [15]:
test_speech = con.sql("select * from speech order by random() limit 1 ").fetchone()

In [20]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

766832
['Herr Beck, wer zuletzt lacht, lacht am besten.', 'Herr Kollege Brandner, wir haben in der Vergangenheit immer dafür gestritten, dass die Mittel zur Verbesserung der regionalen Wirtschaftsstruktur erhöht werden. Heute sind wir dabei, den Haushalt, der eingebracht wurde, in erster Lesung zu beraten.', 'Wir werden ihn weiterhin genau unter die Lupe nehmen. Dabei gehe ich davon aus, dass man meinen Aussagen nachkommt. Ich habe das nicht einfach so gesagt, sondern meine Aussagen sind fundiert. Ich habe sie eruiert und bin zu dem Schluss gekommen, dass das etwas Vernünftiges, Gutes und Richtiges ist. Was die Verbesserung der regionalen Wirtschaftsstruktur angeht, müssen auch die Konversionsmittel, die aufgrund der Schließung verschiedener Bundeswehrstandorte zur Verfügung gestellt werden, einbezogen werden. Ich befürchte, Sie haben sie nicht berücksichtigt und sind deshalb zu anderen Ergebnissen gekommen als ich.', 'Sie dürfen versichert sein, dass ich über meine Fraktion so gut wie

In [16]:
print(test_speech[5])

<sp who="Ernst Hinsken" parliamentary_group="CDU/CSU" role="mp" position="NA" who_original="Ernst Hinsken" party="CSU" name="Ernst Hinsken">
          <speaker>Ernst Hinsken (CDU/CSU):</speaker>
          <p>Herr Beck, wer zuletzt lacht, lacht am besten.</p>
          <p>Herr Kollege Brandner, wir haben in der Vergangenheit immer dafür gestritten, dass die Mittel zur Verbesserung der regionalen Wirtschaftsstruktur erhöht werden. Heute sind wir dabei, den Haushalt, der eingebracht wurde, in erster Lesung zu beraten.</p>
          <stage type="interjection">(Zurufe von der SPD: Ach so! - Aha! - Interessant!)</stage>
          <p>Wir werden ihn weiterhin genau unter die Lupe nehmen. Dabei gehe ich davon aus, dass man meinen Aussagen nachkommt. Ich habe das nicht einfach so gesagt, sondern meine Aussagen sind fundiert. Ich habe sie eruiert und bin zu dem Schluss gekommen, dass das etwas Vernünftiges, Gutes und Richtiges ist. Was die Verbesserung der regionalen Wirtschaftsstruktur angeht, m

In [57]:
print(ET.tostring(root.find('name'), encoding='unicode'))

<name>Vizepräsidentin Katrin Göring-Eckardt:</name>
                
